In [1]:
import sys
import os
from dotenv import load_dotenv
from datetime import datetime
import json
from collections import Counter,defaultdict
import pandas as pd
import random
# Set the seed
seed_value = 15
random.seed(seed_value)

load_dotenv()


# Set the root directory of your project
project_root = '/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_project'

os.chdir(project_root)

# Add the root directory to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
# Read the CSV file into a DataFrame
file_path = "testing/prompts/prompts_all_20241124.csv"  # Replace with your CSV file path
df = pd.read_csv(file_path)

# Apply a filter
# Example: Filter rows where 'column_name' has values greater than a threshold
filtered_df = df[df['Name'] == "Justus"]  # Replace 'column_name' and 'threshold'

# # Display the filtered DataFrame
# print(filtered_df)

# # Optionally, save the filtered data to a new CSV
# filtered_df.to_csv("filtered_file.csv", index=False)

In [3]:
filtered_df['Question'].to_list()

['Gab es im August 2024 eine Förderung für Gashybridheizungen?',
 'Welche Vorlauftemperatur ist maximal zulässig, damit aktuelle Heizflächen gefördert werden?',
 'Welche Vorlauftemperatur ist maximal zulässig, damit aktuelle Heizflächen gefördert werden?',
 'Wann werden Heizflächen gefördert?',
 'Welche technischen Anforderungen bestehen an Heizflächen?',
 'an bei einer neuen Wärmepumpe auch einen eigenen Stromanschluss legen lasen',
 'Welche technischen Anforderungen bestehen an die Beleuchtung',
 'In welchem Maße werden neue Dachflächenfenster in einem Wohnhaus gefördert',
 'Wann werden neue Dachflächenfenster gefördert',
 'welche technischen Anforderungen bestehen an neue Dachflächenfenster, damit sie gefördert werden',
 'Müssen neue Dachflächenfenster in einem bestehenden Wohngebäude einen bestimmten U-Wert erreichen',
 'Wie ist der luftdichte Anschluss bei einer Dampfbremse in der obersten Geschossdecke herzustellen',
 'Welchen U-Wert müssen Dachflächenfenster haben, um gefördert 

In [4]:
from utils.MySQLDB_manager import MySQLDB
from utils.chunking_embedding import process_metadata_csv
from config import SQL_CHUNK_TABLE,CONFIG_SQL_DB,DB_NAME,METADATA_FILE_PATH,SQL_EVAL_QUESTIONS_ANSWERS_TABLE,SQL_EVAL_QUESTIONS_ANSWERS_TABLE_SCHEMA

sql_con = MySQLDB(CONFIG_SQL_DB,DB_NAME)

chunks = sql_con.get_all_records_as_dict(SQL_CHUNK_TABLE)
metadata_df = process_metadata_csv(METADATA_FILE_PATH)

chunks_recent = []
for chunk in chunks:
    chunk['metadata'] = json.loads(chunk['metadata'])
    chunk['doc_type'] = chunk['metadata']['doc_type']
    chunk['source'] = chunk['metadata']['source'].strip()
    recent = metadata_df[metadata_df['file'] == chunk['source']]['most_recent'].iloc[0]
    chunk['metadata'] = json.dumps(chunk['metadata'],ensure_ascii=False)

    if recent:
        chunks_recent.append(chunk)

# Group chunks by doc_type
chunks_by_type = defaultdict(list)
for chunk in chunks_recent:
    chunks_by_type[chunk['doc_type']].append(chunk)

# Display the number of chunks for each doc_type
print("Chunks grouped by type:")
for doc_type, chunks in chunks_by_type.items():
    print(f"{doc_type}: {len(chunks)} chunks")


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rodolfocacacho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Chunks grouped by type:
Richtlinie BEG EM: 64 chunks
Infoblätter förderfähigen Kosten: 54 chunks
FAQ BEG (EM und EH-EG): 64 chunks
Liste förderfähigen Anlagen - Wärmepumpen: 307 chunks
Förderübersicht BEG EM: 2 chunks
Liste förderfähigen Anlagen - Biomasse: 68 chunks
Technische FAQ BEG EM: 80 chunks
Allgemeines Merkblatt zur Antragstellung: 14 chunks


In [5]:
import random
import math

def calculate_sample_size(total_chunks):
    """Calculate sample size based on document size."""
    if total_chunks <= 10:
        return total_chunks
    return math.floor((total_chunks - 10) / 10) + 10

def systematic_sampling(chunks, sample_size):
    step = max(1, len(chunks) // sample_size)
    sampled_indices = range(0, len(chunks), step)
    return [chunks[i] for i in sampled_indices[:sample_size]]

# Example usage
# sample_size = calculate_sample_size(len(chunks))
# sampled_chunks = systematic_sampling(chunks, sample_size)

def sample_chunks(chunks, sample_size, expansion_factor=3):
    """
    Randomly sample chunks while avoiding overlap caused by expansion.
    
    Parameters:
    - chunks: List of chunks to sample from.
    - sample_size: Number of chunks to sample.
    - expansion_factor: Number of adjacent chunks to include for context.
    
    Returns:
    - List of sampled chunks.
    """
    total_chunks = len(chunks)
    sampled_indices = set()
    
    while len(sampled_indices) < sample_size:
        candidate = random.randint(0, total_chunks - 1)
        
        # Ensure no overlap between sampled chunks and their expansion
        if all(
            candidate + offset not in sampled_indices
            for offset in range(-expansion_factor + 1, expansion_factor)
        ):
            sampled_indices.add(candidate)
    
    # Return sampled chunks
    return [chunks[i] for i in sorted(sampled_indices)]

def random_sampling(chunks, sample_size):
    """
    Randomly sample chunks without repetition.
    
    Args:
        chunks (list): The list of chunks to sample from.
        sample_size (int): Number of chunks to sample.
    
    Returns:
        list: The sampled chunks.
    """
    if sample_size > len(chunks):
        raise ValueError("Sample size cannot be greater than the total number of chunks.")
    
    # Randomly sample indices
    sampled_indices = random.sample(range(len(chunks)), sample_size)
    
    # Retrieve the sampled chunks
    return [chunks[i] for i in sampled_indices]

In [16]:
seed_value = 42
random.seed(seed_value)
sampled_chunks_by_type = {}

for doc_type, chunks in chunks_by_type.items():
    total_chunks = len(chunks)
    sample_size = calculate_sample_size(total_chunks)
    if total_chunks <= 10:
        # Include all chunks for small documents
        sampled_chunks_by_type[doc_type] = chunks
    else:
        # Random sampling for larger documents
        sampled_chunks_by_type[doc_type] = random.sample(chunks, sample_size)

In [17]:
for doc_type,docs in sampled_chunks_by_type.items():
    print(len(docs))
    ids = []
    for i in docs:
        ids.append(i['id'])
    print(ids)

15
['2.21', '2.1', '2.51', '2.24', '2.22', '2.9', '2.16', '2.7', '2.14', '2.48', '2.6', '2.4', '2.13', '2.42', '2.33']
14
['20.10', '20.1', '20.13', '20.20', '20.21', '20.38', '20.43', '20.8', '20.40', '20.2', '20.46', '20.4', '20.32', '20.53']
15
['35.60', '35.42', '35.24', '35.55', '35.59', '35.0', '35.52', '35.63', '35.18', '35.49', '35.33', '35.28', '35.7', '35.17', '35.20']
39
['56.253', '56.145', '56.140', '56.273', '56.142', '56.263', '56.257', '56.22', '56.118', '56.34', '56.7', '56.155', '56.272', '56.134', '56.77', '56.233', '56.265', '56.89', '56.187', '56.13', '56.119', '56.202', '56.231', '56.205', '56.144', '56.226', '56.31', '56.266', '56.173', '56.269', '56.261', '56.195', '56.220', '56.130', '56.177', '56.69', '56.210', '56.35', '56.222']
2
['64.0', '64.1']
15
['66.34', '66.46', '66.15', '66.35', '66.12', '66.55', '66.27', '66.31', '66.24', '66.66', '66.20', '66.41', '66.5', '66.64', '66.60']
17
['76.66', '76.54', '76.61', '76.25', '76.39', '76.24', '76.37', '76.73', '

In [18]:
sampled_chunks_by_type

{'Richtlinie BEG EM': [{'id': '2.21',
   'content': 'Bedingung ist der Austausch von funktionstüchtigen Öl-, Kohle-, Gas-Etagen- und Nachtspeicherheizungen (ohne Anforderung an den Zeitpunkt der Inbetriebnahme) oder von funktionstüchtigen Gasheizungen oder Biomasseheizun-gen, wenn die Inbetriebnahme zum Zeitpunkt der Antragstellung mindestens 20 Jahre zurückliegt. Voraussetzung für die Gewährung des Bonus ist eine fachgerechte Demontage und Entsorgung der ausgetauschten, für den Bonus berechtigten Heizung.\nFür die Errichtung von Biomasseheizungen nach Nummer 5.3 Buchstabe b und Buchstabe g wird der Bonus nur gewährt, wenn diese mit einer solarthermischen Anlage oder einer Anlage zur Erzeugung von Strom aus solarer Strahlungsenergie zur elektrischen Warmwasserbereitung oder einer Wärmepumpe zur Warmwasserbereitung und/ oder Raumheizungsunterstützung kombiniert werden. Diese Anlagen sind mindestens so zu dimensionieren, dass sie die Trinkwassererwärmung bilanziell vollständig decken kön

In [20]:
sql_con.create_table(SQL_EVAL_QUESTIONS_ANSWERS_TABLE,SQL_EVAL_QUESTIONS_ANSWERS_TABLE_SCHEMA)

for i,list_sample in sampled_chunks_by_type.items():
    sql_con.insert_many_records(SQL_EVAL_QUESTIONS_ANSWERS_TABLE,list_sample)